In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

import numpy as np


### VERİYİ TANIMA 

In [ ]:
data= pd.read_csv(r'C:\Users\beyza\Downloads\Restaurant_revenue.csv')
data.head()

In [ ]:
data.info()

- DataFrame sınıfına ait bir nesne 
- 1000 satır gözlem içerir . index 0 ile 999 arasındadır ( RangeIndex: 1000 entries , 0 to 999 )
- 8 sütun içerir.

Restoran Geliri Tahmini Veri Kümesi, bir dizi hayali restoran için aylık gelirin tahmin edilmesine odaklanan, tahmine dayalı modelleme çalışmaları için hazırlanmış simüle edilmiş bir veri kümesidir. Bu veri seti, restoran performansını etkileyen faktörleri taklit etmek için tasarlanmış çok çeşitli özellikler sunar.

**Veri Kümesi Özellikleri:**

1. *Number_of_Customers (Müşteri Sayısı)*

- &nbsp; Restoranı ziyaret eden müşterilerin sayısını belirtir.
- &nbsp; Yüksek müşteri sayısı genellikle daha yüksek gelirle ilişkilidir. Müşteri sayısı, restoranın popülaritesini ve müşteri tabanının genişliğini gösterir.

2. *Menu_Price (Menü Fiyatı)*

- &nbsp; Restorandaki ortalama menü fiyatlarını gösterir.
- &nbsp; Menü fiyatları, gelir tahmininde önemli bir faktördür. Yüksek fiyatlar daha yüksek gelir sağlayabilir, ancak müşteri sayısını da etkileyebilir.

3. *Marketing_Spend (Pazarlama Harcaması)*

- &nbsp; Pazarlama faaliyetlerine ilişkin harcamaları belirtir.
- &nbsp; Pazarlama harcamaları, müşteri çekme ve gelir artırma stratejilerinde kritik bir rol oynar. Daha yüksek pazarlama harcamaları genellikle daha fazla müşteri çekebilir.

4. *Cuisine_Type (Mutfak Türü)*

- &nbsp; Restoranın sunduğu mutfağın türünü belirtir (örneğin, İtalyan, Meksika, Japon, Amerikan).
- &nbsp; Mutfak türü, belirli müşteri segmentlerini çekebilir ve restoranın popülaritesini etkileyebilir. Farklı mutfak türleri farklı fiyatlandırma stratejilerine sahip olabilir.
 
5. *Average_Customer_Spending (Ortalama Müşteri Harcaması)*

- &nbsp; Müşteri başına ortalama harcamayı gösterir.
- &nbsp; Ortalama müşteri harcaması, gelir tahmini için kritik bir faktördür. Müşterilerin ne kadar harcadığı, toplam geliri doğrudan etkiler.

6. *Promotions (Promosyonlar)*

- &nbsp; Promosyonların yapılıp yapılmadığını gösteren ikili bir göstergedir (0 veya 1).
- &nbsp; Promosyonlar, müşteri çekmek ve satışları artırmak için yaygın olarak kullanılır. Promosyonların varlığı (1), satışlarda geçici bir artışa neden olabilir.

7. *Reviews (Yorumlar)*

- &nbsp; Restoranın aldığı yorumların sayısını belirtir.
- &nbsp; Yorumlar, restoranın itibarını ve müşteri memnuniyetini yansıtır. Daha fazla olumlu yorum, daha fazla müşteri çekebilir ve geliri artırabilir.

8. *Monthly_Revenue (Aylık Gelir)*

- &nbsp; Restoranın simüle edilmiş aylık gelirini gösterir.
- &nbsp; Bu, tahmin için hedef değişkendir ve yukarıdaki faktörlerin etkilediği sonucu gösterir. Amaç, diğer faktörlere dayanarak bu değeri tahmin etmektir.

---

### Veri Temizleme 
Eksik verilerin ve aykırı değerlerin kontrol edilmesi ve gerektiğinde düzeltilmesi.

In [ ]:
#from pandas
data.isnull().any()

In [ ]:
#eksik veri varsa toplam kaç tane
data.isnull().sum()

**DataFrame'in hiçbir sütununda eksik(null) veri bulunmamaktadır. Veri analizi ve modelleme işlemi için eksik veri imputation (doldurma) veya veri temizleme adımlarına gerek yok.**

---

In [ ]:
data.describe()

In [ ]:
#Temel Istatistikler
print(data.describe().T)

# Keşifsel Veri Analizi (EDA)

#### Outlier ( aykırı) değer tespiti / temizlenmesi

Aykırı değerleri bulmak için genellikle belirli bir eşik değeri kullanılır. Bu eşik değeri, genellikle verinin istatistiksel özelliklerine göre belirlenir. Örneğin, verinin çeyrekliklerine dayalı olarak belirlenen IQR (Interquartile Range) kullanılabilir.

Veri setinizdeki aykırı değerleri bulmak için aşağıdaki adımları izleyebilirsiniz:

Veri setinizdeki her bir özellik için istatistiksel özetleri inceleyin. Bu, her bir özellik için ortalama, standart sapma, minimum, maksimum, çeyreklikler gibi değerleri içerecektir.

Çeyreklikler ve IQR kullanarak bir eşik değeri belirleyin. Örneğin, alt çeyreklik (Q1) ve üst çeyreklik (Q3) arasındaki farkı ifade eden IQR'ı hesaplayın. Ardından, aykırı değerler için alt ve üst sınırı belirlemek için bu IQR'ı kullanın. Genellikle, alt sınır Q1 - 1.5 * IQR ve üst sınır Q3 + 1.5 * IQR olarak belirlenir.

Belirlediğiniz eşik değeri kullanarak, her bir özellik için aykırı değerleri tespit edin. Örneğin, alt sınırdan küçük veya üst sınırdan büyük olan değerler aykırı değer olarak kabul edilir.

--

aykırı değerleri her sütun için ayrı ayrı bulmanız gerekecek çünkü her sütunun istatistiksel özellikleri farklı olabilir ve aykırı değerlerin tespiti için farklı eşik değerlerine ihtiyaç duyulabilir. Bu nedenle, her sütun için ayrı ayrı çeyreklikleri, IQR'ı ve sonrasında aykırı değerleri bulmanız gerekmektedir.

İşte her sütun için aykırı değerleri bulmak için genel bir yol:

In [ ]:
# Her bir sütun için aykırı değerleri bulun
for column in data.columns:
    
    if not pd.api.types.is_numeric_dtype(data[column]):      # Sayısal olmayan sütunları atla
        print("Sayısal olmayan sütun - {}: Aykırı değer hesaplanamaz.\n".format(column))
        continue                                             
    
    # Sütun için çeyreklikleri ve IQR'ı hesaplayın
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1

    # Aykırı değerleri tespit edin
    alt_sinir = Q1 - 1.5 * IQR
    ust_sinir = Q3 + 1.5 * IQR
    aykirilar = (data[column] < alt_sinir) | (data[column] > ust_sinir)

    # Aykırı değerleri yazdırın
    print("Aykırı Değerler - {}: ".format(column))
    print(data[column][aykirilar])
    print("\n")

    # Aykırı değerleri görselleştirin
    plt.figure(figsize=(8, 6))
    plt.hist(data[column], bins=30, color='skyblue', edgecolor='black', alpha=0.7, label='Veri')
    plt.hist(data[column][aykirilar], bins=10, color='red', edgecolor='black', alpha=0.7, label='Aykırı Değerler')
    plt.xlabel(column)
    plt.ylabel('Frekans')
    plt.title('Aykırı Değerlerin Görselleştirilmesi - {}'.format(column))
    plt.legend()
    plt.show()


#### Sürekli ve Kategorik değişkenlerin temizlenmesi

In [ ]:
def check_data_types(df):
    data_types = {}
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            data_types[column] = 'numeric'
        else:
            data_types[column] = 'categorical'
    return data_types



# Veri türlerini kontrol etmek için fonksiyonu çağırma
column_data_types = check_data_types(data)
for column, data_type in column_data_types.items():
    print(f"{column}: {data_type}")


# To Do = gerçekte verilerin numeric mi değil mi olduğunu tartış